# Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# install

In [ ]:
!pip install -U datasets==2.20.0 bitsandbytes accelerate peft==0.10.0 trl==0.9.6 # transformers==4.41.2 다운그레이드 시 Trainer import 호환 오류

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 16.1 MB/s eta 0:00:00
  Using cached 

# 8.3 Reward Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf", split="train[:10000]")
dataset

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 10000
})

In [ ]:
def preprocess(batch):
    result = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for chosen, rejected in zip(batch["chosen"], batch["rejected"]):
        tokenized_chosen = tokenizer(chosen)
        tokenized_rejected = tokenizer(rejected)

        result["input_ids_chosen"].append(
            tokenized_chosen["input_ids"]
        )
        result["attention_mask_chosen"].append(
            tokenized_chosen["attention_mask"]
        )
        result["input_ids_rejected"].append(
            tokenized_rejected["input_ids"]
        )
        result["attention_mask_rejected"].append(
            tokenized_rejected["attention_mask"]
        )

    return result

dataset = dataset.map(
    preprocess,
    batched=True,
    num_proc=2,
    remove_columns=dataset.column_names
)

max_length = 512
dataset = dataset.filter(
    lambda x: (
        len(x["input_ids_chosen"]) <= max_length
        and len(x["input_ids_rejected"]) <= max_length
    )
)

dataset

Dataset({
    features: ['input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
    num_rows: 9701
})

In [ ]:
from trl import RewardTrainer, RewardConfig

config = RewardConfig(
    logging_dir="/content/logs",
    output_dir="/content/ckpt",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,
    report_to="tensorboard",
)

trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=config,
    train_dataset=dataset,
)
trainer.train()

# 8.4 SFT: Supervised Fine-Tuning

## 8.3.1 기본 구조

In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

dataset = load_dataset("imdb", split="train")

sft_config = SFTConfig(
    dataset_text_field="text",
    output_dir="/content/ckpt",
    report_to="tensorboard",
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=sft_config,
)
# trainer.train()  # 실제 학습할때만 사용하기

DatacollatorForCompletionOnlyLM

In [ ]:
from transformers import AutoTokenizer
from trl import DataCollatorForCompletionOnlyLM

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
collator = DataCollatorForCompletionOnlyLM(
    response_template=" [/INST]",
    tokenizer=tokenizer
)

prompt_example = "<s>[INST] this is input prompt [/INST] this is response. is it work?</s>"
example = collator([tokenizer(prompt_example)])

label = example.labels[0]
print(label)
print("only response:", tokenizer.decode(label[label > 0]))

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100,   42,   16, 1263,    4,   16,   24,  173,  116,    2])
only response:  this is response. is it work?</s>


In [ ]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))

prompt = "[INST] this is input prompt [/INST] this is response. is it work?"
print_tokens_with_ids(prompt)

response_template = "[/INST]"
print_tokens_with_ids(response_template)

[('[', 10975), ('INST', 39236), (']', 742), ('Ġthis', 42), ('Ġis', 16), ('Ġinput', 8135), ('Ġprompt', 14302), ('Ġ[/', 48651), ('INST', 39236), (']', 742), ('Ġthis', 42), ('Ġis', 16), ('Ġresponse', 1263), ('.', 4), ('Ġis', 16), ('Ġit', 24), ('Ġwork', 173), ('?', 116)]
[('[/', 48505), ('INST', 39236), (']', 742)]


setup chat format

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format

model_name = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("before:", tokenizer.chat_template)

model, tokenizer = setup_chat_format(model, tokenizer)
print("after:", tokenizer.chat_template)

before: None
after: {% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


formatting func

In [ ]:
from datasets import Dataset


example = [
    {"question": "질문 1", "answer": "답변 1"},
    {"question": "질문 2", "answer": "답변 2"},
    {"question": "질문 3", "answer": "답변 3"},
]
test_dataset = Dataset.from_list(example)

def formatting_prompts_func(sample):
    output_texts = []
    for i in range(len(sample["question"])):
        text = (
            f"### Question: {sample["question"][i]}\n "
            f"### Answer: {sample["answer"][i]}"
        )
        output_texts.append(text)
    return output_texts

sft_config = SFTConfig(
    dataset_text_field="text",
    output_dir="/content/ckpt",
    report_to="tensorboard",
)
trainer = SFTTrainer(
    model,
    args=sft_config,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
)

# trainer.train()  # 실제 학습할때만 사용하기

## packing

In [ ]:
sft_config = SFTConfig(
    packing=True,
    max_seq_length=512,
    dataset_text_field="text",
    output_dir="/content/ckpt",
    report_to="tensorboard",
)

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=sft_config
)

# trainer.train()  # 실제 학습할때만 사용하기

## model_init_kwargs

In [ ]:
sft_config = SFTConfig(
    model_init_kwargs={
        "torch_dtype": "bfloat16",
    },
    max_seq_length=512,
    dataset_text_field="text",
    output_dir="/content/ckpt",
    report_to="tensorboard",
)
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=sft_config,
)

# trainer.train()  # 실제 학습할때만 사용하기

## peft_config

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=SFTConfig(
        max_seq_length=512,
        dataset_text_field="text",
        output_dir="/content/ckpt",
        report_to="tensorboard",
    ),
    peft_config=peft_config
)

# trainer.train()  # 실제 학습할때만 사용하기

In [ ]:
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=SFTConfig(
        max_seq_length=512,
        dataset_text_field="text",
        output_dir="/content/ckpt",
        report_to="tensorboard",
    ),
    model_init_kwargs={
        "torch_dtype": "bfloat16",
        "load_in_4bit": True,
    },
    peft_config=peft_config,
)

# trainer.train()  # 실제 학습할때만 사용하기

## model_config

In [ ]:
from trl import (
    ModelConfig,
    SFTTrainer,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
)

model_config = ModelConfig(
    model_name_or_path="facebook/opt-350m",
    load_in_4bit=True,
    use_peft=True,
)
model_config

ModelConfig(model_name_or_path='facebook/opt-350m', model_revision='main', torch_dtype=None, trust_remote_code=False, attn_implementation=None, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, lora_target_modules=None, lora_modules_to_save=None, lora_task_type='CAUSAL_LM', load_in_8bit=False, load_in_4bit=True, bnb_4bit_quant_type='nf4', use_bnb_nested_quant=False)

In [ ]:
quantization_config = get_quantization_config(model_config)
quantization_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float32",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [ ]:
get_kbit_device_map()

{'': 0}

In [ ]:
peft_config = get_peft_config(model_config)
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [ ]:
torch_dtype = (
    model_config.torch_dtype
    if model_config.torch_dtype in ["auto", None]
    else getattr(torch, model_config.torch_dtype)
)

model_kwargs = dict(
    revision=model_config.model_revision,
    trust_remote_code=model_config.trust_remote_code,
    attn_implementation=model_config.attn_implementation,
    torch_dtype=torch_dtype,
    device_map=(
        get_kbit_device_map()
        if quantization_config is not None
        else None
    ),
    quantization_config=quantization_config.to_dict(),
)

training_args = SFTConfig(
    max_seq_length=512,
    dataset_text_field="text",
    output_dir="/content/ckpt",
    report_to="tensorboard",
    model_init_kwargs=model_kwargs,
)

trainer = SFTTrainer(
    model=model_config.model_name_or_path,
    train_dataset=dataset,
    args=training_args,
    peft_config=peft_config,
)

# trainer.train()  # 실제 학습할때만 사용하기

## neftune_noise_alpha

In [ ]:
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    args=SFTConfig(
        max_seq_length=512,
        dataset_text_field="text",
        output_dir="/content/ckpt",
        report_to="tensorboard",
        neftune_noise_alpha=5,
    ),
)
# trainer.train()  # 실제 학습할때만 사용하기

# 8.5 PPO: Proximal Policy Optimization


In [ ]:
# 사용법 확인 용도이므로 제대로 학습된 보상 모델 사용 x.
# KLD 값이 음수로 떨어져 다수의 warning 메세지 발생. 이를 삭제.
import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
from transformers import AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

config = PPOConfig(
    model_name="gpt2",
    learning_rate=1.41e-5,
    mini_batch_size=1,
    batch_size=1,
    report_to="tensorboard",
)
model = (
    AutoModelForCausalLMWithValueHead.from_pretrained(
        config.model_name
    )
)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

reward_model = pipeline(
    "text-classification",
    model="lvwerra/distilbert-imdb"
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "HuggingFaceH4/cherry_picked_prompts",
    split="train",
)
print(dataset)
dataset[0]

Generating train split:   0%|          | 0/16 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'completion', 'meta'],
    num_rows: 16
})


{'prompt': 'Explain the moon landing to a 6 year old in a few sentences.',
 'completion': 'People went to the moon, and they took pictures of what they saw, and sent them back to the earth so we could all see them.',
 'meta': {'source': 'instructgpt'}}

In [ ]:
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["query"])
    return sample

dataset = dataset.rename_column("prompt", "query")
dataset = dataset.remove_columns(["meta", "completion"])
dataset = dataset.map(tokenize, batched=False)
dataset.set_format(type="torch")

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
generation_kwargs = {
    "max_length": 400,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

dataset = dataset.filter(
    lambda x: len(x["input_ids"]) <= generation_kwargs["max_length"]
)

Filter:   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
from trl import PPOTrainer

ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset,
    tokenizer=tokenizer,
    data_collator=collator,
)

In [ ]:
import torch
from tqdm.notebook import tqdm


epochs = 10
for epoch in tqdm(range(epochs), "epoch: "):
    for batch in tqdm(ppo_trainer.dataloader, leave=False):
        query_tensors = batch["input_ids"]

        # Rollout: 학습할 모델로 문장 생성
        response_tensors = ppo_trainer.generate(
            query_tensors,
            **generation_kwargs
        )

        batch["response"] = [
            tokenizer.decode(r.squeeze())
            for r in response_tensors
        ]

        # Evaluate: Reward 모델로 점수 부여
        # return_full_text 옵션 입력이 불가능하므로, response == query + gen_text
        try:
            pipe_outputs = reward_model(batch["response"])
        except:
            print([r.size() for r in response_tensors])
            print(reward_model.tokenizer(batch["response"], return_length=True)["length"])

        rewards = [
            torch.tensor(output["score"])
            for output in pipe_outputs
        ]

        # Optimization: ppo 학습 진행
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

# 모델 저장
ppo_trainer.save_pretrained("/content/my_ppo_model")

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

Token indices sequence length is longer than the specified maximum sequence length for this model (6528 > 512). Running this sequence through the model will result in indexing errors


[torch.Size([400])]
[6528]


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

# 8.6 Best of N Sampling

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, GenerationConfig
from trl import AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
from trl.extras import BestOfNSampler

ref_model_name = "gpt2"
reward_model_name = "gpt2"
device = torch.device("cuda")
ref_model = (
    AutoModelForCausalLMWithValueHead
    .from_pretrained(ref_model_name)
    .to(device)
)
tokenizer = AutoTokenizer.from_pretrained(ref_model_name)

reward_pipe = pipeline(
    "sentiment-analysis",
    model=reward_model_name,
    device=device
)

def queries_to_scores(list_of_strings):
    return [output["score"] for output in reward_pipe(list_of_strings)]

best_of_n = BestOfNSampler(
    ref_model,
    tokenizer,
    queries_to_scores,
    length_sampler=LengthSampler(10, 128),
    sample_size=5,
    n_candidates=2,
    generation_config=GenerationConfig(
        min_length= -1,
        top_k=0.0,
        top_p= 1.0,
        do_sample= True,
        pad_token_id=tokenizer.eos_token_id
    ),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
result = best_of_n.generate(
    tokenizer("what is love?", return_tensors="pt").input_ids[0],
    device=device
)

for r in result[0]:
    print(r)
    print("=" * 50, "\n")

what is love? And think empire? No, I wouldn't spoil it for you."

"That's something the Lannisters already knew…" "It was suggested!"

"Oh, quite." I grin now, knowing straight away that he has let his secrets be sold on.

"No," he says as he breathlessly leaves. "Try going to Kreg, see how this new Rosser and Shepard

what is love?

They more like sport than art. And there is no heaven and no earth! Just pump me the water.

And I must ask: Where does my love live?"

(Bram Stoker 1997)

With that, Staker comes to us again. To raise you up because you're queer, because you are queer, and something was broken. Oh, what kind of breaking



# 8.7 DPO: Directi Preference Optimization


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
ref_model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="bfloat16")
ref_model = AutoModelForCausalLM.from_pretrained(
    ref_model_name, 
    torch_dtype="bfloat16",
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-internal-testing/hh-rlhf-trl-style",
    split="train[:10000]",
)
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 10000
})

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.chat_template is None:
    tokenizer.chat_template = (
        "{% for message in messages %}"
        "{{message['role'] + ': ' + message['content'] + '\n\n'}}"
        "{% endfor %}{{ eos_token }}"
    )

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(
        row["chosen"], tokenize=False
    )
    row["rejected"] = tokenizer.apply_chat_template(
        row["rejected"], tokenize=False
    )
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)

print(dataset[0]["chosen"])

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

user: What are some cuss words in english?

assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

user: What's your favorite one?

assistant: I haven't even thought about it.

<|endoftext|>


In [ ]:
from trl import DPOConfig, DPOTrainer

args = DPOConfig(
    beta=0.1,
    max_length=512,
    max_prompt_length=512,
    dataset_num_proc=2,
    remove_unused_columns=False,
    output_dir="/content/ckpt",
    report_to="tensorboard",
)

trainer = DPOTrainer(
    model,
    ref_model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=args,
)

trainer.train()

# 8.8 KTO: Kahneman-Tversky Optimization

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "facebook/opt-350m"
ref_model_name = "facebook/opt-350m"

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="bfloat16")
ref_model = AutoModelForCausalLM.from_pretrained(ref_model_name, torch_dtype="bfloat16")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import load_dataset

dataset = load_dataset("trl-lib/kto-mix-14k", split="train")
dataset

Dataset({
    features: ['prompt', 'completion', 'label'],
    num_rows: 13500
})

In [ ]:
def process(row):
    row["prompt"] = tokenizer.apply_chat_template(
        row["prompt"], tokenize=False
    )
    row["completion"] = tokenizer.apply_chat_template(
        row["completion"], tokenize=False
    )
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)
print(dataset[0]["completion"])

Map (num_proc=2):   0%|          | 0/13500 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


 Yes, the information you found on Google is correct. Julio César Chávez holds several records related to world title defenses and victories, and he is considered one of the greatest boxers in history. Here is a detailed answer to your question:

Julio César Chávez was born on July 12, 1962, in Ciudad Obregón, Sonora, Mexico. He began boxing at a young age and quickly made a name for himself in the sport, winning his first world title in 1984 when he defeated Mario Miranda for the WBC super featherweight championship.

Over the course of his career, Chávez would go on to hold titles in three different weight classes (super featherweight, lightweight, and junior welterweight) and defend his titles a record 27 times, including 21 times by knockout. This record for most successful consecutive defenses of world titles has never been broken, and it is a testament to Chávez's skill, determination, and durability as a fighter.

In addition to his record for most successful title defenses, Chá

In [ ]:
from trl import KTOTrainer, KTOConfig

args = KTOConfig(
    logging_dir="/content/logs",
    output_dir="/content/ckpt",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,
    report_to="tensorboard",

    max_length=512,
    max_prompt_length=512,
    remove_unused_columns=False,
    dataset_num_proc=2,

    beta=0.1,
    desirable_weight=1.0,
    undesirable_weight=1.0,
)

trainer = KTOTrainer(
    model,
    ref_model,
    args=args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

trainer.train()

# 8.9 CPO: Contrastive Preference Optimization

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-internal-testing/hh-rlhf-trl-style",
    split="train[:10000]",
)
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 10000
})

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.chat_template is None:
    tokenizer.chat_template = (
        "{% for message in messages %}"
        "{{message['role'] + ': ' + message['content'] + '\n\n'}}"
        "{% endfor %}{{ eos_token }}"
    )

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(
        row["chosen"], tokenize=False
    )
    row["rejected"] = tokenizer.apply_chat_template(
        row["rejected"], tokenize=False
    )
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from trl import CPOConfig, CPOTrainer

args = CPOConfig(
    logging_dir="/content/logs",
    output_dir="/content/ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,
    report_to="tensorboard",

    max_length=512,
    max_prompt_length=512,
    dataset_num_proc=2,
    remove_unused_columns=False,
    beta=0.1,
)

trainer = CPOTrainer(
    model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Token indices sequence length is longer than the specified maximum sequence length for this model (1029 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1047 > 1024). Running this sequence through the model will result in indexing errors
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
100,5.699500
200,4.772200


KeyboardInterrupt: 

# 8.10 ORPO: Odds Ratio Preference Optimization

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="bfloat16")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-internal-testing/hh-rlhf-trl-style",
    split="train[:10000]",
)
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 10000
})

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.chat_template is None:
    tokenizer.chat_template = (
        "{% for message in messages %}"
        "{{message['role'] + ': ' + message['content'] + '\n\n'}}"
        "{% endfor %}{{ eos_token }}"
    )

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(
        row["chosen"], tokenize=False
    )
    row["rejected"] = tokenizer.apply_chat_template(
        row["rejected"], tokenize=False
    )
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from trl import ORPOConfig, ORPOTrainer

args = ORPOConfig(
    logging_dir="/content/logs",
    output_dir="/content/ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,
    report_to="tensorboard",

    max_length=512,
    max_prompt_length=512,
    dataset_num_proc=2,
    remove_unused_columns=False,

    beta=0.1,
)

trainer = ORPOTrainer(
    model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=dataset,
)

trainer.train()